In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("Data_for_KEA.xlsx")
df

,Organization,Title (Project's name),Project's price (DKK),How we got the lead?,Deal created on,Won time,Contact person,Creator,Deal closed on,Email messages count,Lost reason,Owner,Status,Total activities
0,Customer#:1,Adding Austrian language deal,22510.80,NaN,2018-08-09 17:10:16,2018-08-10 14:56:09,Contact person#:2,Andrius,2018-08-10 14:56:09,0,NaN,Andrius,Won,0
1,Customer#:2,"Site, speed improvements",25012.00,NaN,2017-01-26 08:49:17,NaN,Contact person#:3,Christian Petersen,2017-01-30 12:40:57,0,Hun fik en kræmmerdeal,Christian Petersen,Lost,0
2,Customer#:3,- Help with school assignment,845.00,Others (random calls / emails),2020-05-18 09:38:27,NaN,Contact person#:4,Dominik,2020-06-09 10:05:00,1,We gave up (poked but got no reply),Dominik,Lost,0
3,Customer#:3,Support Voucher,29363.75,It is an existing client,2021-09-13 13:09:39,NaN,Contact person#:5,Joakim,2021-09-29 14:08:02,0,Price was too high,Joakim,Lost,0
4,Customer#:4,Workshop,42250.00,NaN,2017-02-09 09:29:18,NaN,Contact person#:6,Bjarke Larsen,2017-02-23 10:07:49,0,Der var ikke nok penge og/eller commitment,Bjarke Larsen,Lost,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3654,Customer#:1242,New Website deal,67600.00,Simon's leads (Ras.Aagaard@html24.net),2019-05-05 08:18:37,NaN,Contact person#:2045,Rasmus,2019-11-18 09:28:41,30,We couldn't reach the decision maker,Rasmus,Lost,3
3655,Customer#:1243,"-, new website",929500.00,NaN,2017-10-26 15:26:23,NaN,Contact person#:2046,Bo Møller,2018-06-25 10:15:40,67,Client chose other agency (elaborate why?),Daniel,Lost,0
3656,Customer#:1244,Conversion API deal,15632.50,Google,2021-02-24 13:01:47,NaN,Contact person#:2047,Rasmus,2021-02-24 15:13:50,9,Client has implemented the solution with inter...,Christian,Lost,0
3657,Customer#:1245,Konsulent deal 1 mrd.,189280.00,NaN,2017-02-09 09:05:29,NaN,Contact person#:2047,Rasmus,2017-03-29 11:35:48,0,Client shut down the project.,Rasmus,Lost,0


In [3]:
df["Project\'s price (DKK)"].nunique()

1124

Vi tjækker for duplicates, for at sikre er ikke er nogle i datasættet

Eftersom der ikke var nogle duplicates eftersom row number er det sammme gør vi ikke mere. 

In [4]:
df["Project\'s price (DKK)"].value_counts()[84500]


184

In [5]:
df.drop_duplicates()

,Organization,Title (Project's name),Project's price (DKK),How we got the lead?,Deal created on,Won time,Contact person,Creator,Deal closed on,Email messages count,Lost reason,Owner,Status,Total activities
0,Customer#:1,Adding Austrian language deal,22510.80,NaN,2018-08-09 17:10:16,2018-08-10 14:56:09,Contact person#:2,Andrius,2018-08-10 14:56:09,0,NaN,Andrius,Won,0
1,Customer#:2,"Site, speed improvements",25012.00,NaN,2017-01-26 08:49:17,NaN,Contact person#:3,Christian Petersen,2017-01-30 12:40:57,0,Hun fik en kræmmerdeal,Christian Petersen,Lost,0
2,Customer#:3,- Help with school assignment,845.00,Others (random calls / emails),2020-05-18 09:38:27,NaN,Contact person#:4,Dominik,2020-06-09 10:05:00,1,We gave up (poked but got no reply),Dominik,Lost,0
3,Customer#:3,Support Voucher,29363.75,It is an existing client,2021-09-13 13:09:39,NaN,Contact person#:5,Joakim,2021-09-29 14:08:02,0,Price was too high,Joakim,Lost,0
4,Customer#:4,Workshop,42250.00,NaN,2017-02-09 09:29:18,NaN,Contact person#:6,Bjarke Larsen,2017-02-23 10:07:49,0,Der var ikke nok penge og/eller commitment,Bjarke Larsen,Lost,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3654,Customer#:1242,New Website deal,67600.00,Simon's leads (Ras.Aagaard@html24.net),2019-05-05 08:18:37,NaN,Contact person#:2045,Rasmus,2019-11-18 09:28:41,30,We couldn't reach the decision maker,Rasmus,Lost,3
3655,Customer#:1243,"-, new website",929500.00,NaN,2017-10-26 15:26:23,NaN,Contact person#:2046,Bo Møller,2018-06-25 10:15:40,67,Client chose other agency (elaborate why?),Daniel,Lost,0
3656,Customer#:1244,Conversion API deal,15632.50,Google,2021-02-24 13:01:47,NaN,Contact person#:2047,Rasmus,2021-02-24 15:13:50,9,Client has implemented the solution with inter...,Christian,Lost,0
3657,Customer#:1245,Konsulent deal 1 mrd.,189280.00,NaN,2017-02-09 09:05:29,NaN,Contact person#:2047,Rasmus,2017-03-29 11:35:48,0,Client shut down the project.,Rasmus,Lost,0


Nu sletter vi de column vi ikke kan bruge her er "Total activities" en column som ikke giver os meget viden og vi sletter derfor denne.

In [6]:
df = df.drop(columns = "Total activities")
df

,Organization,Title (Project's name),Project's price (DKK),How we got the lead?,Deal created on,Won time,Contact person,Creator,Deal closed on,Email messages count,Lost reason,Owner,Status
0,Customer#:1,Adding Austrian language deal,22510.80,NaN,2018-08-09 17:10:16,2018-08-10 14:56:09,Contact person#:2,Andrius,2018-08-10 14:56:09,0,NaN,Andrius,Won
1,Customer#:2,"Site, speed improvements",25012.00,NaN,2017-01-26 08:49:17,NaN,Contact person#:3,Christian Petersen,2017-01-30 12:40:57,0,Hun fik en kræmmerdeal,Christian Petersen,Lost
2,Customer#:3,- Help with school assignment,845.00,Others (random calls / emails),2020-05-18 09:38:27,NaN,Contact person#:4,Dominik,2020-06-09 10:05:00,1,We gave up (poked but got no reply),Dominik,Lost
3,Customer#:3,Support Voucher,29363.75,It is an existing client,2021-09-13 13:09:39,NaN,Contact person#:5,Joakim,2021-09-29 14:08:02,0,Price was too high,Joakim,Lost
4,Customer#:4,Workshop,42250.00,NaN,2017-02-09 09:29:18,NaN,Contact person#:6,Bjarke Larsen,2017-02-23 10:07:49,0,Der var ikke nok penge og/eller commitment,Bjarke Larsen,Lost
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3654,Customer#:1242,New Website deal,67600.00,Simon's leads (Ras.Aagaard@html24.net),2019-05-05 08:18:37,NaN,Contact person#:2045,Rasmus,2019-11-18 09:28:41,30,We couldn't reach the decision maker,Rasmus,Lost
3655,Customer#:1243,"-, new website",929500.00,NaN,2017-10-26 15:26:23,NaN,Contact person#:2046,Bo Møller,2018-06-25 10:15:40,67,Client chose other agency (elaborate why?),Daniel,Lost
3656,Customer#:1244,Conversion API deal,15632.50,Google,2021-02-24 13:01:47,NaN,Contact person#:2047,Rasmus,2021-02-24 15:13:50,9,Client has implemented the solution with inter...,Christian,Lost
3657,Customer#:1245,Konsulent deal 1 mrd.,189280.00,NaN,2017-02-09 09:05:29,NaN,Contact person#:2047,Rasmus,2017-03-29 11:35:48,0,Client shut down the project.,Rasmus,Lost


vi kan se at Title har nogle special tegn som vi skal have slettet 

In [7]:
df["Title (Project's name)"] = df["Title (Project's name)"].str.strip(",-=  //")
df

,Organization,Title (Project's name),Project's price (DKK),How we got the lead?,Deal created on,Won time,Contact person,Creator,Deal closed on,Email messages count,Lost reason,Owner,Status
0,Customer#:1,Adding Austrian language deal,22510.80,NaN,2018-08-09 17:10:16,2018-08-10 14:56:09,Contact person#:2,Andrius,2018-08-10 14:56:09,0,NaN,Andrius,Won
1,Customer#:2,"Site, speed improvements",25012.00,NaN,2017-01-26 08:49:17,NaN,Contact person#:3,Christian Petersen,2017-01-30 12:40:57,0,Hun fik en kræmmerdeal,Christian Petersen,Lost
2,Customer#:3,Help with school assignment,845.00,Others (random calls / emails),2020-05-18 09:38:27,NaN,Contact person#:4,Dominik,2020-06-09 10:05:00,1,We gave up (poked but got no reply),Dominik,Lost
3,Customer#:3,Support Voucher,29363.75,It is an existing client,2021-09-13 13:09:39,NaN,Contact person#:5,Joakim,2021-09-29 14:08:02,0,Price was too high,Joakim,Lost
4,Customer#:4,Workshop,42250.00,NaN,2017-02-09 09:29:18,NaN,Contact person#:6,Bjarke Larsen,2017-02-23 10:07:49,0,Der var ikke nok penge og/eller commitment,Bjarke Larsen,Lost
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3654,Customer#:1242,New Website deal,67600.00,Simon's leads (Ras.Aagaard@html24.net),2019-05-05 08:18:37,NaN,Contact person#:2045,Rasmus,2019-11-18 09:28:41,30,We couldn't reach the decision maker,Rasmus,Lost
3655,Customer#:1243,new website,929500.00,NaN,2017-10-26 15:26:23,NaN,Contact person#:2046,Bo Møller,2018-06-25 10:15:40,67,Client chose other agency (elaborate why?),Daniel,Lost
3656,Customer#:1244,Conversion API deal,15632.50,Google,2021-02-24 13:01:47,NaN,Contact person#:2047,Rasmus,2021-02-24 15:13:50,9,Client has implemented the solution with inter...,Christian,Lost
3657,Customer#:1245,Konsulent deal 1 mrd.,189280.00,NaN,2017-02-09 09:05:29,NaN,Contact person#:2047,Rasmus,2017-03-29 11:35:48,0,Client shut down the project.,Rasmus,Lost


In [13]:
df["Deal created on"] = pd.to_datetime(df["Deal created on"])
df["Won time"] = pd.to_datetime(df["Won time"])
df["Deal closed on"] = pd.to_datetime(df["Deal closed on"])
df

,ID,Organization,Title (Project's name),Project's price (DKK),How we got the lead?,Deal created on,Won time,Contact person,Creator,Deal closed on,Email messages count,Lost reason,Owner,Status
0,0,Customer#:1,Adding Austrian language deal,22510.80,,2018-08-09 17:10:16,2018-08-10 14:56:09,Contact person#:2,Andrius,2018-08-10 14:56:09,0,,Andrius,Won
1,1,Customer#:2,"Site, speed improvements",25012.00,,2017-01-26 08:49:17,NaT,Contact person#:3,Christian Petersen,2017-01-30 12:40:57,0,Hun fik en kræmmerdeal,Christian Petersen,Lost
2,2,Customer#:3,Help with school assignment,845.00,Others (random calls / emails),2020-05-18 09:38:27,NaT,Contact person#:4,Dominik,2020-06-09 10:05:00,1,We gave up (poked but got no reply),Dominik,Lost
3,3,Customer#:3,Support Voucher,29363.75,It is an existing client,2021-09-13 13:09:39,NaT,Contact person#:5,Joakim,2021-09-29 14:08:02,0,Price was too high,Joakim,Lost
4,4,Customer#:4,Workshop,42250.00,,2017-02-09 09:29:18,NaT,Contact person#:6,Bjarke Larsen,2017-02-23 10:07:49,0,Der var ikke nok penge og/eller commitment,Bjarke Larsen,Lost
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3654,3654,Customer#:1242,New Website deal,67600.00,Simon's leads (Ras.Aagaard@html24.net),2019-05-05 08:18:37,NaT,Contact person#:2045,Rasmus,2019-11-18 09:28:41,30,We couldn't reach the decision maker,Rasmus,Lost
3655,3655,Customer#:1243,new website,929500.00,,2017-10-26 15:26:23,NaT,Contact person#:2046,Bo Møller,2018-06-25 10:15:40,67,Client chose other agency (elaborate why?),Daniel,Lost
3656,3656,Customer#:1244,Conversion API deal,15632.50,Google,2021-02-24 13:01:47,NaT,Contact person#:2047,Rasmus,2021-02-24 15:13:50,9,Client has implemented the solution with inter...,Christian,Lost
3657,3657,Customer#:1245,Konsulent deal 1 mrd.,189280.00,,2017-02-09 09:05:29,NaT,Contact person#:2047,Rasmus,2017-03-29 11:35:48,0,Client shut down the project.,Rasmus,Lost


In [16]:
df['Days_to_win'] = (df["Won time"] - df["Deal created on"]).dt.days
df['Days_to_close'] = (df["Deal closed on"] - df["Deal created on"]).dt.days
df

,ID,Organization,Title (Project's name),Project's price (DKK),How we got the lead?,Deal created on,Won time,Contact person,Creator,Deal closed on,Email messages count,Lost reason,Owner,Status,Days_to_win,Days_to_close
0,0,Customer#:1,Adding Austrian language deal,22510.80,,2018-08-09 17:10:16,2018-08-10 14:56:09,Contact person#:2,Andrius,2018-08-10 14:56:09,0,,Andrius,Won,0.0,0.0
1,1,Customer#:2,"Site, speed improvements",25012.00,,2017-01-26 08:49:17,NaT,Contact person#:3,Christian Petersen,2017-01-30 12:40:57,0,Hun fik en kræmmerdeal,Christian Petersen,Lost,NaN,4.0
2,2,Customer#:3,Help with school assignment,845.00,Others (random calls / emails),2020-05-18 09:38:27,NaT,Contact person#:4,Dominik,2020-06-09 10:05:00,1,We gave up (poked but got no reply),Dominik,Lost,NaN,22.0
3,3,Customer#:3,Support Voucher,29363.75,It is an existing client,2021-09-13 13:09:39,NaT,Contact person#:5,Joakim,2021-09-29 14:08:02,0,Price was too high,Joakim,Lost,NaN,16.0
4,4,Customer#:4,Workshop,42250.00,,2017-02-09 09:29:18,NaT,Contact person#:6,Bjarke Larsen,2017-02-23 10:07:49,0,Der var ikke nok penge og/eller commitment,Bjarke Larsen,Lost,NaN,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3654,3654,Customer#:1242,New Website deal,67600.00,Simon's leads (Ras.Aagaard@html24.net),2019-05-05 08:18:37,NaT,Contact person#:2045,Rasmus,2019-11-18 09:28:41,30,We couldn't reach the decision maker,Rasmus,Lost,NaN,197.0
3655,3655,Customer#:1243,new website,929500.00,,2017-10-26 15:26:23,NaT,Contact person#:2046,Bo Møller,2018-06-25 10:15:40,67,Client chose other agency (elaborate why?),Daniel,Lost,NaN,241.0
3656,3656,Customer#:1244,Conversion API deal,15632.50,Google,2021-02-24 13:01:47,NaT,Contact person#:2047,Rasmus,2021-02-24 15:13:50,9,Client has implemented the solution with inter...,Christian,Lost,NaN,0.0
3657,3657,Customer#:1245,Konsulent deal 1 mrd.,189280.00,,2017-02-09 09:05:29,NaT,Contact person#:2047,Rasmus,2017-03-29 11:35:48,0,Client shut down the project.,Rasmus,Lost,NaN,48.0


Vi sletter NaN i datasættet

In [17]:
df = df.fillna("")
df

,ID,Organization,Title (Project's name),Project's price (DKK),How we got the lead?,Deal created on,Won time,Contact person,Creator,Deal closed on,Email messages count,Lost reason,Owner,Status,Days_to_win,Days_to_close
0,0,Customer#:1,Adding Austrian language deal,22510.80,,2018-08-09 17:10:16,2018-08-10 14:56:09,Contact person#:2,Andrius,2018-08-10 14:56:09,0,,Andrius,Won,0.0,0.0
1,1,Customer#:2,"Site, speed improvements",25012.00,,2017-01-26 08:49:17,NaT,Contact person#:3,Christian Petersen,2017-01-30 12:40:57,0,Hun fik en kræmmerdeal,Christian Petersen,Lost,,4.0
2,2,Customer#:3,Help with school assignment,845.00,Others (random calls / emails),2020-05-18 09:38:27,NaT,Contact person#:4,Dominik,2020-06-09 10:05:00,1,We gave up (poked but got no reply),Dominik,Lost,,22.0
3,3,Customer#:3,Support Voucher,29363.75,It is an existing client,2021-09-13 13:09:39,NaT,Contact person#:5,Joakim,2021-09-29 14:08:02,0,Price was too high,Joakim,Lost,,16.0
4,4,Customer#:4,Workshop,42250.00,,2017-02-09 09:29:18,NaT,Contact person#:6,Bjarke Larsen,2017-02-23 10:07:49,0,Der var ikke nok penge og/eller commitment,Bjarke Larsen,Lost,,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3654,3654,Customer#:1242,New Website deal,67600.00,Simon's leads (Ras.Aagaard@html24.net),2019-05-05 08:18:37,NaT,Contact person#:2045,Rasmus,2019-11-18 09:28:41,30,We couldn't reach the decision maker,Rasmus,Lost,,197.0
3655,3655,Customer#:1243,new website,929500.00,,2017-10-26 15:26:23,NaT,Contact person#:2046,Bo Møller,2018-06-25 10:15:40,67,Client chose other agency (elaborate why?),Daniel,Lost,,241.0
3656,3656,Customer#:1244,Conversion API deal,15632.50,Google,2021-02-24 13:01:47,NaT,Contact person#:2047,Rasmus,2021-02-24 15:13:50,9,Client has implemented the solution with inter...,Christian,Lost,,0.0
3657,3657,Customer#:1245,Konsulent deal 1 mrd.,189280.00,,2017-02-09 09:05:29,NaT,Contact person#:2047,Rasmus,2017-03-29 11:35:48,0,Client shut down the project.,Rasmus,Lost,,48.0


Så tilføjer vi en ID column til datasættet

Nu gemmer vi den nye version

In [18]:
df.reset_index(inplace=True)
df.rename(columns={'index': 'ID'}, inplace=True)
df

,ID,ID,Organization,Title (Project's name),Project's price (DKK),How we got the lead?,Deal created on,Won time,Contact person,Creator,Deal closed on,Email messages count,Lost reason,Owner,Status,Days_to_win,Days_to_close
0,0,0,Customer#:1,Adding Austrian language deal,22510.80,,2018-08-09 17:10:16,2018-08-10 14:56:09,Contact person#:2,Andrius,2018-08-10 14:56:09,0,,Andrius,Won,0.0,0.0
1,1,1,Customer#:2,"Site, speed improvements",25012.00,,2017-01-26 08:49:17,NaT,Contact person#:3,Christian Petersen,2017-01-30 12:40:57,0,Hun fik en kræmmerdeal,Christian Petersen,Lost,,4.0
2,2,2,Customer#:3,Help with school assignment,845.00,Others (random calls / emails),2020-05-18 09:38:27,NaT,Contact person#:4,Dominik,2020-06-09 10:05:00,1,We gave up (poked but got no reply),Dominik,Lost,,22.0
3,3,3,Customer#:3,Support Voucher,29363.75,It is an existing client,2021-09-13 13:09:39,NaT,Contact person#:5,Joakim,2021-09-29 14:08:02,0,Price was too high,Joakim,Lost,,16.0
4,4,4,Customer#:4,Workshop,42250.00,,2017-02-09 09:29:18,NaT,Contact person#:6,Bjarke Larsen,2017-02-23 10:07:49,0,Der var ikke nok penge og/eller commitment,Bjarke Larsen,Lost,,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3654,3654,3654,Customer#:1242,New Website deal,67600.00,Simon's leads (Ras.Aagaard@html24.net),2019-05-05 08:18:37,NaT,Contact person#:2045,Rasmus,2019-11-18 09:28:41,30,We couldn't reach the decision maker,Rasmus,Lost,,197.0
3655,3655,3655,Customer#:1243,new website,929500.00,,2017-10-26 15:26:23,NaT,Contact person#:2046,Bo Møller,2018-06-25 10:15:40,67,Client chose other agency (elaborate why?),Daniel,Lost,,241.0
3656,3656,3656,Customer#:1244,Conversion API deal,15632.50,Google,2021-02-24 13:01:47,NaT,Contact person#:2047,Rasmus,2021-02-24 15:13:50,9,Client has implemented the solution with inter...,Christian,Lost,,0.0
3657,3657,3657,Customer#:1245,Konsulent deal 1 mrd.,189280.00,,2017-02-09 09:05:29,NaT,Contact person#:2047,Rasmus,2017-03-29 11:35:48,0,Client shut down the project.,Rasmus,Lost,,48.0


In [19]:
df.to_excel("cleanedv2_data_for_KEA.xlsx", index = False)
